# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 15 2022 8:30AM,244107,10,MSP213078,"Methapharm, Inc.",Released
1,Aug 15 2022 8:30AM,244107,10,MSP213079,"Methapharm, Inc.",Released
2,Aug 15 2022 8:30AM,244107,10,MSP213080,"Methapharm, Inc.",Released
3,Aug 15 2022 8:30AM,244107,10,MSP213081,"Methapharm, Inc.",Released
4,Aug 15 2022 8:30AM,244107,10,MSP213082,"Methapharm, Inc.",Released
5,Aug 15 2022 8:30AM,244107,10,MSP213083,"Methapharm, Inc.",Released
6,Aug 15 2022 8:30AM,244107,10,MSP213084,"Methapharm, Inc.",Released
7,Aug 15 2022 8:30AM,244107,10,MSP213088,"Methapharm, Inc.",Released
8,Aug 15 2022 8:30AM,244107,10,MSP213089,"Methapharm, Inc.",Released
9,Aug 15 2022 8:30AM,244107,10,MSP213090,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,244093,Released,1
25,244100,Released,1
26,244101,Released,13
27,244107,Released,10
28,244108,Released,11


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244093,NaN,NaN,1.0
244100,NaN,NaN,1.0
244101,NaN,NaN,13.0
244107,NaN,NaN,10.0
244108,NaN,NaN,11.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244014,0.0,0.0,1.0
244023,0.0,0.0,12.0
244024,3.0,4.0,17.0
244026,9.0,1.0,24.0
244036,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244014,0,0,1
244023,0,0,12
244024,3,4,17
244026,9,1,24
244036,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244014,0,0,1
1,244023,0,0,12
2,244024,3,4,17
3,244026,9,1,24
4,244036,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244014,,,1
1,244023,,,12
2,244024,3,4,17
3,244026,9,1,24
4,244036,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc."
10,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation
21,Aug 12 2022 3:29PM,244101,20,Alumier Labs Inc.
34,Aug 12 2022 2:35PM,244100,20,Sartorius Stedim North America
35,Aug 12 2022 2:05PM,244093,21,"NBTY Global, Inc."
36,Aug 12 2022 2:05PM,244092,21,"NBTY Global, Inc."
37,Aug 12 2022 2:03PM,244090,21,"NBTY Global, Inc."
38,Aug 12 2022 2:01PM,244088,10,"Methapharm, Inc."
44,Aug 12 2022 2:01PM,244086,10,"Methapharm, Inc."
63,Aug 12 2022 12:03PM,244069,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc.",,,10
1,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation,,,11
2,Aug 12 2022 3:29PM,244101,20,Alumier Labs Inc.,,,13
3,Aug 12 2022 2:35PM,244100,20,Sartorius Stedim North America,,,1
4,Aug 12 2022 2:05PM,244093,21,"NBTY Global, Inc.",,,1
5,Aug 12 2022 2:05PM,244092,21,"NBTY Global, Inc.",,,1
6,Aug 12 2022 2:03PM,244090,21,"NBTY Global, Inc.",,,1
7,Aug 12 2022 2:01PM,244088,10,"Methapharm, Inc.",,,6
8,Aug 12 2022 2:01PM,244086,10,"Methapharm, Inc.",,,19
9,Aug 12 2022 12:03PM,244069,20,Alumier Labs Inc.,,,8


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc.",10,,
1,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation,11,,
2,Aug 12 2022 3:29PM,244101,20,Alumier Labs Inc.,13,,
3,Aug 12 2022 2:35PM,244100,20,Sartorius Stedim North America,1,,
4,Aug 12 2022 2:05PM,244093,21,"NBTY Global, Inc.",1,,
5,Aug 12 2022 2:05PM,244092,21,"NBTY Global, Inc.",1,,
6,Aug 12 2022 2:03PM,244090,21,"NBTY Global, Inc.",1,,
7,Aug 12 2022 2:01PM,244088,10,"Methapharm, Inc.",6,,
8,Aug 12 2022 2:01PM,244086,10,"Methapharm, Inc.",19,,
9,Aug 12 2022 12:03PM,244069,20,Alumier Labs Inc.,8,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc.",10,,
1,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation,11,,
2,Aug 12 2022 3:29PM,244101,20,Alumier Labs Inc.,13,,
3,Aug 12 2022 2:35PM,244100,20,Sartorius Stedim North America,1,,
4,Aug 12 2022 2:05PM,244093,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc.",10.0,NaN,NaN
1,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation,11.0,NaN,NaN
2,Aug 12 2022 3:29PM,244101,20,Alumier Labs Inc.,13.0,NaN,NaN
3,Aug 12 2022 2:35PM,244100,20,Sartorius Stedim North America,1.0,NaN,NaN
4,Aug 12 2022 2:05PM,244093,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc.",10.0,0.0,0.0
1,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation,11.0,0.0,0.0
2,Aug 12 2022 3:29PM,244101,20,Alumier Labs Inc.,13.0,0.0,0.0
3,Aug 12 2022 2:35PM,244100,20,Sartorius Stedim North America,1.0,0.0,0.0
4,Aug 12 2022 2:05PM,244093,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1708508,76.0,4.0,3.0
102,244014,1.0,0.0,0.0
15,244036,1.0,0.0,0.0
16,1220287,4.0,1.0,0.0
20,1464435,77.0,20.0,18.0
21,732275,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1708508,76.0,4.0,3.0
1,102,244014,1.0,0.0,0.0
2,15,244036,1.0,0.0,0.0
3,16,1220287,4.0,1.0,0.0
4,20,1464435,77.0,20.0,18.0
5,21,732275,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,76.0,4.0,3.0
1,102,1.0,0.0,0.0
2,15,1.0,0.0,0.0
3,16,4.0,1.0,0.0
4,20,77.0,20.0,18.0
5,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,76.0
1,102,Released,1.0
2,15,Released,1.0
3,16,Released,4.0
4,20,Released,77.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,16,20,21
Status,,,,,,
Completed,3.0,0.0,0.0,0.0,18.0,0.0
Executing,4.0,0.0,0.0,1.0,20.0,0.0
Released,76.0,1.0,1.0,4.0,77.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,16,20,21
0,Completed,3.0,0.0,0.0,0.0,18.0,0.0
1,Executing,4.0,0.0,0.0,1.0,20.0,0.0
2,Released,76.0,1.0,1.0,4.0,77.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,16,20,21
0,Completed,3.0,0.0,0.0,0.0,18.0,0.0
1,Executing,4.0,0.0,0.0,1.0,20.0,0.0
2,Released,76.0,1.0,1.0,4.0,77.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()